# Assignment 1: Tokenization and Word counts for sentiment analysis
In this assignment, you will be applying the techniques learned in week 1 of the course to perform and analyze sentiment on a dataset of movie reviews from IMDB.

This dataset comes from [Mass et. al. (2011)](https://www.aclweb.org/anthology/P11-1015.pdf) and the full version is available [here](http://ai.stanford.edu/~amaas/data/sentiment/).

In [14]:
# setup
import sys
import subprocess
import pkg_resources
from collections import Counter
import re
from numpy import log, mean

required = {'spacy', 'scikit-learn', 'pandas', 'transformers==2.4.1'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import spacy
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer

## Read in data
I've saved a subset of the data in the data directory on the repository.  It is available as a pickled dictionary.


In [23]:
from glob import glob
import numpy as np
pct_sample = 0.1
all_text = {}
for p in ['neg', 'pos']:
    all_text[p] = []
    for f in glob('/Users/batorsky/Downloads/aclImdb/test/%s/*.txt' % p):
        if np.random.rand()<=pct_sample:
            all_text[p].append(open(f, encoding='utf-8').read())


In [ ]:
with open('../data/assignment_1_reviews.pkl', 'wb') as f:
    pickle.dump(all_text, f)

In [20]:
# you will need to change this to where ever the file is stored
data_location = '../data/assignment_1_reviews.pkl'
with open(data_location, 'rb') as f:
    all_text = pickle.load(f)
# corpora size
print([(k, len(all_text[k])) for k in all_text])
# for simplicity, let's split these into separate sets
neg, pos = all_text.values()

[('neg', 1233), ('pos', 1266)]


## Tokenization
Use what you've developed in the week 1 notebook to tokenize each of the corpora.

In [2]:
from spacy.lang.en import English
en = English()

def simple_tokenizer(doc, model=en):
    # a simple tokenizer for individual documents (different from above)
    tokenized_docs = []
    parsed = model(doc)
    return([t.lower_ for t in parsed if (t.is_alpha)&(not t.like_url)])

In [5]:
token_neg = [simple_tokenizer(x) for x in neg]
token_pos = [simple_tokenizer(x) for x in pos]

## Word counts
Create a count of the number of words in each review.  Use scikit-learn's CountVectorizer.  Refer to the documentation as it has a few parameters you might want to think about.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(tokenizer=simple_tokenizer)
# should probably fit on the combined
cv.fit(neg+pos)
count_neg = cv.transform(neg).toarray()
count_pos = cv.transform(pos).toarray()

In [12]:
# can use pandas DF here
neg_df = pd.DataFrame(count_neg, columns=cv.get_feature_names())
neg_df['positive'] = False
pos_df = pd.DataFrame(count_pos, columns=cv.get_feature_names())
pos_df['positive'] = True
# combine for more ease
all_df = pos_df.append(neg_df)

In [13]:
all_df.head()

,a,aa,aaaaaaahhhhhhggg,aaaggghhhhhhh,aaah,aaam,aaja,aalcc,aaliyah,aaran,...,zwart,zweite,à,é,étienne,être,ïts,über,ý,ýs
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Most frequent words
What are the top 10 most frequent words in the positive reviews? The negative reviews?

In [56]:
def get_top(data, n=10):
    return(
        data.groupby('positive').sum().T.apply(lambda c: c.nlargest(n)))

In [57]:
get_top(all_df)

positive,False,True
a,7548.0,8133.0
and,6978.0,8905.0
i,4248.0,3943.0
in,4203.0,5093.0
is,4952.0,5793.0
it,4354.0,4553.0
of,6402.0,7942.0
that,NaN,3521.0
the,15365.0,17182.0
this,3837.0,NaN


It seems like there's a lot of pretty irrelevant words in the top here.  It's hard to really say anything about this.  Can you think of a way to get to more informative terms (i.e. ones that might give you some insight as to what words are positive versus negative?)

Hint: Think about which tokens might be less informative.  Is there a way we learned to remove those?

In [17]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [33]:
minus_stop = all_df.iloc[:, ~all_df.columns.isin(ENGLISH_STOP_WORDS)]

In [70]:
get_top(all_df.iloc[:, ~all_df.columns.isin(ENGLISH_STOP_WORDS)], n=90).reset_index().values

array([['acting', 369.0, 236.0],
       ['action', 154.0, 155.0],
       ['actors', 265.0, 220.0],
       ['actually', 253.0, 163.0],
       ['awful', 147.0, nan],
       ['bad', 668.0, 194.0],
       ['beautiful', nan, 157.0],
       ['best', 187.0, 440.0],
       ['better', 298.0, 221.0],
       ['big', 162.0, 179.0],
       ['bit', nan, 171.0],
       ['book', 142.0, nan],
       ['boring', 149.0, nan],
       ['br', 412.0, 404.0],
       ['ca', 192.0, 168.0],
       ['cast', 172.0, 220.0],
       ['character', 364.0, 374.0],
       ['characters', 365.0, 370.0],
       ['come', 149.0, nan],
       ['comedy', 155.0, 161.0],
       ['course', nan, 154.0],
       ['did', 637.0, 423.0],
       ['director', 212.0, 240.0],
       ['does', 521.0, 508.0],
       ['dvd', nan, 185.0],
       ['end', 265.0, 251.0],
       ['especially', nan, 166.0],
       ['excellent', nan, 183.0],
       ['fact', 179.0, 165.0],
       ['family', nan, 192.0],
       ['far', 165.0, 149.0],
       ['feel', nan,

Check how often the top words from negative appear in the positive reviews and vice versa.  Do these seem like good candidates for determining whether a review is positive or negative? If not, maybe expand to the top 10, or more.  The idea here is to get a list of terms that are pretty distinct between the two sets.

Extra: Look into [this resource](https://wordhoard.northwestern.edu/userman/analysis-comparewords.html) for a way to test the count difference.

In [80]:
def log_likelihood(data, word):
    counts = data.groupby('positive').sum()
    a = counts.loc[True, word]
    b = counts.loc[False, word]
    c = counts.loc[True].sum()
    d = counts.loc[False].sum()
    print('counts neg:', b)
    print('counts pos:', a)
    e1 = c*(a+b)/(c+d)
    e2 = d*(a+b)/(c+d)
    g = 2*((a*log(a/e1)) + (b*log(b/e2)))
    print('G2: ', g)

In [68]:
# the above gives us some candidates
# function to do likelihood ratio test
words_to_try = ['good', 'character', 'story', 'acting']
for w in words_to_try:
    print(w)
    log_likelihood(all_df, w)

good
counts neg: 632
counts pos: 811
G2:  11.479911369377177
character
counts neg: 364
counts pos: 374
G2:  0.3417317394041106
story
counts neg: 485
counts pos: 625
G2:  9.249181555354653
acting
counts neg: 369
counts pos: 236
G2:  39.552133175346455


## Dictionary-based sentiment analysis 
Construct a list of the keywords you've found are good determinants if a review is positive or negative.  Use this list to "score" a review based on the number of times that word appears in the review.

(Optional) A quick and fancy way of doing this is to use CountVectorizer's vocabulary parameter.  Think how you might be able to do that.

In [71]:
pos_vocab = ['good', 'great', 'best', 'love', 'story']
neg_vocab = ['bad', 'worst', 'acting', 'poor']
sentiment_cv = CountVectorizer(tokenizer=simple_tokenizer, vocabulary=pos_vocab+neg_vocab)
sentiment_cv.fit(neg+pos)

/opt/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function simple_tokenizer at 0x10a661bf8>,
                vocabulary=['good', 'great', 'best', 'love', 'story', 'bad',
                            'worst', 'acting', 'poor'])

How did you do? How often do the negative reviews have a higher negative score than a positive score?

In [ ]:
# average score
neg_sentiment_df = pd.DataFrame(sentiment_cv.transform(neg).toarray(),
                      columns=sentiment_cv.get_feature_names())
pos_sentiment_df = pd.DataFrame(sentiment_cv.transform(pos).toarray(),
                      columns=sentiment_cv.get_feature_names())
print('% of negative reviews with higher neg score:', 
      mean(neg_sentiment_df[neg_vocab].sum(axis=1)>neg_sentiment_df[pos_vocab].sum(axis=1)))
print('% of positive reviews with higher pos score:', 
      mean(pos_sentiment_df[pos_vocab].sum(axis=1)>pos_sentiment_df[neg_vocab].sum(axis=1)))

## Model-based sentiment analysis
Above we did some tinkering with our scoring and found it works to some extent, but it's likely not going to work the same on another dataset.  That is, it's not particularly generalizable.  However, modern sentiment analysis has moved away from dictionary-based scoring towards having sentiment be a "classification" problem.  

For this last section, take a look at the transformers [Pipelines](https://github.com/huggingface/transformers#quick-tour-of-pipelines) functionality.  You'll see that with a few lines of code you can bring in an advanced sentiment analysis model.  Run this against the positive/negative corpus and see how it works compared to your work above.

In [17]:
from transformers import pipeline
nlp = pipeline('sentiment-analysis')

In [21]:
neg_parsed = [nlp(d) for d in neg]

In [22]:
pos_parsed = [nlp(d) for d in pos]

In [29]:
# using % labelled negative vs positive
print('% neg reviews labelled negative:', 
      mean([doc[0]['label']=='NEGATIVE' for doc in neg_parsed]))
print('% pos reviews labelled positive:', 
      mean([doc[0]['label']=='POSITIVE' for doc in pos_parsed]))

% neg reviews labelled negative: 0.9180859691808597
% pos reviews labelled positive: 0.8593996840442338
